In [ ]:
from time import sleep
from pynq import Overlay
ps_gpio_design = Overlay("./bitstream/mybitstream.bit")
from pynq import GPIO

In [ ]:
### Initialisation
# opcode	done	out	        in2	    in1	    in0
# 51—49	    48	    47—————24	23——16	15——8	7——0

opcode_pins = list(range(49, 52))
done_pins = 48
out_pins = list(range(24, 47))
in2_pins = list(range(16, 23))
in1_pins = list(range(8, 15))
in0_pins = list(range(0, 7))

opcode = [GPIO(GPIO.get_gpio_pin(pin), 'out') for pin in opcode_pins]
done = GPIO(GPIO.get_gpio_pin(done_pins), 'in')
out = [GPIO(GPIO.get_gpio_pin(pin), 'in') for pin in out_pins]
in2 = [GPIO(GPIO.get_gpio_pin(pin), 'out') for pin in in2_pins]
in1 = [GPIO(GPIO.get_gpio_pin(pin), 'out') for pin in in1_pins]
in0 = [GPIO(GPIO.get_gpio_pin(pin), 'out') for pin in in0_pins]

In [ ]:
# Opcode 0: Add
opcode_val = 0
in1_val = 3
in2_val = 4

opcode_val[2].write(0)
opcode_val[1].write(0)
opcode_val[0].write(0)


In [ ]:
def write_bus(val, bus):
    for i, g in enumerate(bus):
        g.write((val >> i) & 1)

def read_bus(bus):
    val = 0
    for i, g in enumerate(bus):
        val |= (g.read() & 1) << i
    return val

def run_test(opcode_val, in0_val=0, in1_val=0, in2_val=0, label=""):
    # Werte maskieren
    in0_val &= 0xFF
    in1_val &= 0xFF
    in2_val &= 0xFF
    # Auf Pins schreiben
    write_bus(opcode_val, opcode)
    write_bus(in0_val, in0)
    write_bus(in1_val, in1)
    write_bus(in2_val, in2)
    sleep(0.01)  # kurze Wartezeit
    # Ergebnisse einlesen
    result = read_bus(out)
    done_val = done.read()
    print(f"{label}: out={result} done={done_val}")

NameError: name 'opcode' is not defined

In [ ]:
# --- Testsuite ---
print("Starte Testbench...")

# Add (opcode 0): in0 + in1
run_test(0b000, in0_val=10, in1_val=5, label="Add 10+5")

# Sub (opcode 1): in0 - in1
run_test(0b001, in0_val=10, in1_val=3, label="Sub 10-3")

# Mult (opcode 2): in0 * in1
run_test(0b010, in0_val=4, in1_val=5, label="Mult 4*5")

# Shift Right (opcode 3): in0 >> in1
run_test(0b011, in0_val=64, in1_val=2, label="Shift Right 64>>2")

# Square (opcode 4): in0^2
run_test(0b100, in0_val=7, label="Square 7^2")

# Cube (opcode 5): in0^3
run_test(0b101, in0_val=3, label="Cube 3^3")

# Triple Add (opcode 6): in0+in1+in2
run_test(0b110, in0_val=2, in1_val=3, in2_val=4, label="Triple Add 2+3+4")

# F(x,y,z) (opcode 7) – Beispiel mit 4-Bit Eingängen
run_test(0b111, in0_val=0b1010, in1_val=0b0101, in2_val=0b0011, label="F(x,y,z)")

In [ ]:
# XOR_input_A.write(0)
# XOR_input_B.write(0)
# print(f"XOR_output_C: {XOR_output_C.read()}")
# XOR_input_A.write(1)
# XOR_input_B.write(0)
# print(f"XOR_output_C: {XOR_output_C.read()}")
# XOR_input_A.write(0)
# XOR_input_B.write(1)
# print(f"XOR_output_C: {XOR_output_C.read()}")
# XOR_input_A.write(1)
# XOR_input_B.write(1)
# print(f"XOR_output_C: {XOR_output_C.read()}")